In [ ]:
!pip install -U openai-whisper
!pip install srt
!pip install openai

In [ ]:
import subprocess
import whisper
from datetime import timedelta
from srt import Subtitle
import srt
import openai
import logging

# 音声データ抽出 (音声ファイル名変更)
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

# Whisperモデル読み込み
model = whisper.load_model("medium")

# 音声ファイル読み込み
audio = whisper.load_audio("audio.mp3")

# 音声認識 (開始・終了時間付きセグメント取得)
result = model.transcribe(audio, verbose=True)
segments = result["segments"]

# SRTファイル用の空リスト初期化
subtitles = []

# セグメントごとにSRT行を作成
for index, segment in enumerate(segments, start=1):
    start_time = timedelta(seconds=segment["start"])
    end_time = timedelta(seconds=segment["end"])
    text = segment["text"]

    subtitle = Subtitle(
        index=index,
        start=start_time,
        end=end_time,
        content=text
    )
    subtitles.append(subtitle)

# SRTファイル内容の作成
srt_content = srt.compose(subtitles)

# ファイルへの書き込み (エンコーディング指定)
with open("output.srt", "w", encoding="utf-8") as f:
    f.write(srt_content)

# SRTファイル読み込み (エンコーディング指定)
with open('output.srt', 'r', encoding='utf-8') as file:
    srt_data = file.read()

In [ ]:
#  ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# OpenAIのAPIキーを設定します
openai.api_key = 'your_api_keys'

def gpt_translate(text):
    try:
        completions = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            temperature=0.2,
            messages=[
                {"role": "user", "content": f"[translate to Japanese]:\n{text}"}
            ],
        )
        return completions.choices[0].message.content
    except Exception as e:
        logging.error(f"Translation failed for '{text}': {e}")
        return text  # 翻訳失敗時は元のテキストをそのまま使用

def read_srt(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        srt_data = file.read()
    return list(srt.parse(srt_data))

def translate_subtitles(subtitles):
    for subtitle in subtitles:
        original_text = subtitle.content
        translated_text = gpt_translate(original_text)
        subtitle.content = translated_text
        logging.info(f"Translated: {original_text} -> {translated_text}")

def write_srt(output_file, subtitles):
    translated_srt_content = srt.compose(subtitles)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(translated_srt_content)

def translate_srt(input_file, output_file):
    subtitles = read_srt(input_file)
    translate_subtitles(subtitles)
    write_srt(output_file, subtitles)

translate_srt('output3.srt', 'translated_file.srt')


In [4]:
# 改行関数 (SRT形式保持)
def add_line_breaks_srt(srt_text):
    lines = srt_text.split("\n")
    result = ""

    for i, line in enumerate(lines):
        if line.strip().isdigit():
            result += line
        elif "-->" in line:
            result += line
        else:
            result += add_line_breaks(line) + "\n"

        # 空行を追加 (字幕間の空白)
        if i < len(lines) - 1 and lines[i + 1] != "" and line != "":
            result += "\n"

    return result.strip()

# 改行関数 (句読点・句点での改行)
def add_line_breaks(text):
    result = ""
    line_length = 0
    limit_length = 10

    for char in text:
        if char == "、" and line_length >= limit_length:
            result += "\n"
            line_length = 0
        elif char == "。":
            result += "\n"
            line_length = 0
        else:
            result += char
            line_length += 1

    return result

# 改行されたテキストを取得
with open('translated_file.srt', 'r', encoding='utf-8') as file:
    srt_text = file.read()

# 改行されたテキストを取得
result = add_line_breaks_srt(srt_text)

# ファイルへの書き込み (エンコーディング指定)
with open("translated_file.srt", "w", encoding='utf-8') as file:
    file.write(result)


In [ ]:
import subprocess

subprocess.run([
    'ffmpeg',
    '-hwaccel', 'auto',  # ハードウェアアクセラレーションを有効にする
    '-i', 'output.mp4',
    '-vf', "subtitles=translated_file.srt:force_style='FontName=Helvetica,FontSize=12'",
    '-c:v', 'h264_amf',  # AMDのハードウェアエンコーダーを利用
    '-c:a', 'copy',  # オーディオストリームをコピーする
    'translated_subtitle.mp4'
])
